<a href="https://colab.research.google.com/github/h30e07/E4sosei/blob/main/2nen3nen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# similarityの算出方法

$$
similarity = {\sum_{w_a\in{(A-A\cap{B})}}}\;{\sum_{w_b\in{(B-A\cap{B})}}}f(w_a, w_b)
\\A:シラバスAに現れる全単語の集合,
\\B:シラバスBに現れる全単語の集合,
\\f(w_a, w_b):単語w_aと単語w_bの関連度合いを-1～1の値で返す関数,
$$


# Google Drive マウント
（一番最初にこれを実行してください）

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# メインプログラムに使用している関数一覧 
(メインプログラムを実行する前にこれらの関数を全て実行してください。
これらの関数全てにおいて、左にチェックマークが付けばOKです）

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def show_distribution(list1):
  """
  分布を可視化（グラフ）する関数

  Parameters
  ----------
  list1 : list
    単語同士の全組み合わせとその関連性が格納されたリスト
  """
  
  output = {}
  similarity_list = [d.get('similarity') for d in list1]
  bins = np.arange(-1, 1, 0.1) # 等差数列
  sr = pd.Series(similarity_list)
  sr1 = pd.cut(sr, bins=bins)
  vc = sr1.value_counts(sort=False)
  y, ind, pacthes = plt.hist(sr,bins = bins.size-2, range =(-1,1))
  print('y', y)
  print('ind', ind)
  plt.show()
  return y.tolist()

In [ ]:
def get_similarity(combination_list, sa_words_count, sb_words_count):
  """
  Parameters
  ----------
  combination_list : list
    単語同士の全組み合わせとその関連性が格納されたリスト
  sa_words_count : int
    シラバスaの前処理後の単語数
  sb_words_count : int
    シラバスbの前処理後の単語数
  """
  
  output = 0
  for  combination in combination_list:
    output += combination['similarity']
  output /= (sa_words_count * sb_words_count)
  return output

In [ ]:
def get_combination_list(sa_words, sb_words):
  """
  Parameters
  ----------
  sa_words : list
    シラバスaの前処理後の単語リスト
  sb_words : list
    シラバスbの前処理後の単語リスト
  """
  
  output = []
  for a in sa_words:
    for b in sb_words:
      try:
        output.append({'word1': a,'word2': b, 'similarity': float(w2v_model.similarity(a, b))})
      except KeyError:
         pass
  return output

In [ ]:
def get_excerpted_info(combination_list):
  """
  combination_listはあまりにもデータ数が多く見ずらいため、
  それから大事な情報を抜粋した辞書を返す。
  excerpted 抜粋した。

  Parameters
  ----------
  combination_list : list
    単語同士の全組み合わせとその関連性が格納されたリスト
  """

  output = {
      'combination_sum': len(combination_list),
      'min_similarity_value': combination_list[-1]['similarity'],
      'max_similarity_value': combination_list[0]['similarity'],
      'distribution': show_distribution(combination_list)
  }
  return output

In [ ]:
def syllabus_similarity(sa_path, sb_path):
  """
  メイン関数。
  入力の2つのファイルから求められたあらゆるデータは変数outputに格納される。

  Parameters
  ----------
  sa_path : string
    シラバスaの形態素解析済みのファイルのパス
  sb_path : string
    シラバスbの形態素解析済みのファイルのパス
  """

  output = {
    'subject_combination': [sa_path, sb_path],
    'similarity': 0,
    'excerpted_info': [],
    'combination_list': [],
  }
  sa = {
      'path': sa_path,
      'words': [],
      'words_set': {},
      'remained_set': {},
  }
  sb = {
      'path': sb_path,
      'words': [],
      'words_set': {},
      'remained_set': {},
  }
  with open(sa['path']) as s1:
    sa['words'] = s1.read()
  
  with open(sb['path']) as s2:
    sb['words'] = s2.read()
  
  # 前処理
  sa['words_set'] = set(sa['words'].split())
  sb['words_set'] = set(sb['words'].split())
  sa['remained_set'] = sa['words_set'] - (sa['words_set'] & sb['words_set'])
  sb['remained_set'] = sb['words_set'] - (sa['words_set'] & sb['words_set'])
  print('sa', sa['remained_set'])
  print('sb', sb['remained_set'])

  # 本処理
  similarity_sum = 0
  if len(sa['remained_set']) != 0  and len(sb['remained_set']) != 0:   # 2つが部分集合出ないとき
    output['combination_list'] = get_combination_list(sa['remained_set'], sb['remained_set']) # combination_listに単語組み合わせとその単語同士の関連性を代入 ここではまだsimilarityの計算をしない
    output['combination_list'] = sorted(output['combination_list'], key=lambda x: x['similarity'], reverse=True)
    output['similarity'] = get_similarity(output['combination_list'], len(sa['remained_set']), len(sb['remained_set'])) # combination_listからsimilarityを求める
    output['excerpted_info'] = get_excerpted_info(output['combination_list'])
  else:
    output['similarity'] = 1
  
  print('output', output)
  return output

# メインプログラム　AかつBの単語を取り除く(0かける)
（準備ができたらこれを実行してください）


In [ ]:
from gensim.models.word2vec import Word2Vec
import glob
from google.colab import files
import csv
import json

if __name__ == '__main__':
  output = []
  w2v_model_path = '/content/drive/MyDrive/sosei/Pre-Trained Word2Vec Models/latest-ja-word2vec-gensim-model/word2vec.gensim.model'
  syllabus_path = [
    '/content/drive/MyDrive/sosei/meishi/meishi_3電気回路ⅠA(2078).txt',
    '/content/drive/MyDrive/sosei/meishi/meishi_3電気回路ⅠB(2079).txt',
    '/content/drive/MyDrive/sosei/meishi/meishi_2微分積分学ⅠA(0018).txt'
  ]
  w2v_model = Word2Vec.load(w2v_model_path)
  output.append(syllabus_similarity(syllabus_path[0], syllabus_path[0]))
  output.append(syllabus_similarity(syllabus_path[0], syllabus_path[1]))
  output.append(syllabus_similarity(syllabus_path[0], syllabus_path[2]))

  ### このプログラムを実行して得られた全てのデータをファイル出力 ###
  with open(f"/content/drive/MyDrive/sosei/result_data/v2.json", 'w') as f:
    json.dump(output, f, indent=2, ensure_ascii=False)